In [1]:
    """
    This code generates a fesibility area for deriving intertidal bathymetry. 
    It computes the area between the the Lowest Astronomical Tide (LAT) and 
    the Mean Seal Level + 10 m, based on a LAT map, and a Bathymetry and 
    elevation GEBCO map. 

        Author: Mario.FuentesMonjaraz@deltares.nl
    """

'\nThis code generates a fesibility area for deriving intertidal bathymetry. \nIt computes the area between the the Lowest Astronomical Tide (LAT) and \nthe Mean Seal Level + 10 m, based on a LAT map, and a Bathymetry and \nelevation GEBCO map. \n\n    Author: Mario.FuentesMonjaraz@deltares.nl\n'

### Define packages

In [22]:
import os
import dask
import xarray as xr
import pyproj
import rioxarray
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape


### Define functions

In [3]:
def assign_projection(ds, epsg=None):

    if not epsg == None:
        proj = pyproj.CRS.from_epsg(int(epsg))
    else:
        proj = pyproj.CRS.from_epsg(int(ds.crs.values.tolist()))

    print(proj,"projection was assigned to the dataset attributes")
    ds.attrs['crs'] = proj
    return ds

def print_ds_properties(rds,epsg=None):
    # Print the grid size
    print("Grid size:", rds.rio.resolution())

    #Print null data
    print("no data:", rds.rio.nodata)

    # Print the projection information
    if rds.rio.crs == None:
        print("There is no projection")
        proj = pyproj.CRS.from_epsg(epsg)
        rds.attrs['crs'] = proj
        rds.rio.set_crs(proj, inplace=True)
    else:
        print("There is projection available")
    
    print("Projection EPSG code is:", rds.rio.crs, "\n")
    return

def change_resolution(ds, new_resolution):
    # Reproject the rioxarray object to the new resolution
    reprojected_ds = ds.rio.reproject(ds.rio.crs, resolution=new_resolution, resampling="bilinear")
    return reprojected_ds

def match_resolution(rds, rds_source):
    # Reproject the rioxarray object to the new resolution
    reprojected_ds = rds.rio.reproject(rds_source.rio.crs, resolution=rds_source.rio.resolution(), resampling= rioxarray.enums.Resampling.bilinear)
    return reprojected_ds

def redefine_null_for_nan(ds, new_null_value):
    # Replace NaN values with the new null value
    ds.values[np.isnan(ds.values)] = new_null_value
    # ds.rio.update({'nodata': new_null_value})
    return ds

def create_gdf_from_geojson_files(input_aoi_data):
    geojson_files = []

    for filename in os.listdir(input_aoi_data):
        if filename.endswith(".geojson"):
            gdf = gpd.read_file(os.path.join(input_aoi_data,filename))
            aoi_id = filename.split('_')
            aoi_id = aoi_id[-1].split('.')[0]
            gdf.insert(1, "aoi", aoi_id)
            geojson_files.append((gdf))

    aoi_gdf = gpd.GeoDataFrame(pd.concat(geojson_files, ignore_index=True)).drop(columns=["id"])
    return aoi_gdf

def clip_raster(rds, geometry):
    rds_clipped = rds.rio.clip(geometry)
    return rds_clipped

def clip_raster_with_gdf(rds, gdf):
    rds_clipped_list = []
    for index, row in gdf.iterrows():
        aoi = row["aoi"]
        try:
            geometry = gdf.iloc[index:index+1].geometry
            rds_clipped = rds.rio.clip(geometry)
            rds_clipped_list.append(rds_clipped)
            print(f"Successful processing row {index} {aoi}")
        except Exception as e:
            print(f"Error processing row {index} {aoi}: {e}")
            rds_clipped_list.append("NaN")
            continue
    print("\n")
    return rds_clipped_list 


def apply_lat_mask(raster1, raster2):
    # Read the raster data
    data1 = raster1.values
    data2 = raster2.values
    
    # Create a new array for the result raster
    result_data = np.full_like(data1, fill_value=np.nan, dtype='float32')
    
    # Compare pixel values and assign new values
    result_data[(data1 >= data2)] = 7
    result_data[(data1 < data2)] = 9
    
    # Create a new rioxarray dataset for the result raster
    result_raster = raster1.copy(data=result_data)
    
    return result_raster


def apply_gebco_mask(raster1, raster2):
    # Read the raster data
    data1 = raster1.values
    data2 = raster2.values
    
    # Create a new array for the result raster
    result_data = np.full_like(data1, fill_value=np.nan, dtype='float32')
    
    # Compare pixel values and assign new values
    result_data[(data2 > 10)] = 11
    
    # Create a new rioxarray dataset for the result raster
    result_raster = raster1.copy(data=result_data)
    
    return result_raster

def apply_gebco_mask(gebco_rds,lat_mask):

    binary_mask = gebco_rds > 10
    lat_mask_gebco_plus_10 = lat_mask.where(~binary_mask, other=11)

    return lat_mask_gebco_plus_10 

### Define packages paths

In [9]:
repository_path = os.path.dirname(os.getcwd())
input_data_path = os.path.join(repository_path,"Data")
input_aoi_data = r"p:\11209821-cmems-global-sdb\00_miscellaneous\AOIs"
output_data_path = os.path.join(repository_path,"Ouput")

if not os.path.exists(output_data_path):
    print("Output data path does not exist. Creating directory...")
    os.makedirs(output_data_path)
    print("Output data path created:", output_data_path)
else:
    print("Input data path already exists:", output_data_path)

Input data path already exists: d:\Projects\Copernicus\Ouput


### Retrieve input data

In [11]:
# gebco_ds = xr.open_dataset(os.path.join(input_data_path,'gebco_2023_n90.0_s0.0_w0.0_e90.0_depthmsl.nc')) 
depthmsl_ds = xr.open_dataset(os.path.join(input_data_path,'gebco_2023_n90.0_s0.0_w0.0_e90.0_depthmsl.nc'), chunks={'lat': 100, 'lon':100})
depthmsl_rds = rds = rioxarray.open_rasterio(os.path.join(input_data_path,'gebco_2023_n90.0_s0.0_w0.0_e90.0_depthmsl.nc'), chunks={'y': 100, 'x':100})

lat_ds = xr.open_dataset(os.path.join(input_data_path,'gebco_2023_n90.0_s0.0_w0.0_e90.0_lat.nc'), chunks={'lat': 100, 'lon':100})
lat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,'gebco_2023_n90.0_s0.0_w0.0_e90.0_lat.nc'), chunks={'y': 100, 'x':100})

lw_ds = xr.open_dataset(os.path.join(input_data_path,'LandWater15ARC.nc'), chunks={'lat': 100, 'lon':100})
lw_rds = rioxarray.open_rasterio(os.path.join(input_data_path,'LandWater15ARC.nc'), chunks={'y': 100, 'x':100})

gebco_ds = xr.open_dataset(os.path.join(input_data_path,'gebco_2023_n90.0_s0.0_w0.0_e90.0.tif'), chunks={'lat': 100, 'lon':100})
gebco_rds = rioxarray.open_rasterio(os.path.join(input_data_path,'gebco_2023_n90.0_s0.0_w0.0_e90.0.tif'), chunks={'y': 100, 'x':100})

# AOIs for al the sites
aoi_gdf = create_gdf_from_geojson_files(input_aoi_data)

# AOI for Sao Paulo
aoi_saopaulo = aoi_gdf[aoi_gdf['aoi'] == "SaoPaulo"]
aoi_saopaulo = aoi_gdf.iloc[0:1]
aoi_saopaulo.reset_index(drop=True, inplace=True)

# AOI for Okha
aoi_wadden_okha = aoi_gdf[aoi_gdf['aoi'] == "Okha"]
aoi_wadden_okha = aoi_gdf.iloc[3:5]
aoi_wadden_okha.reset_index(drop=True, inplace=True)

# Bounding boxes of the areas of interes
aoi_gdf_bounding_box  = aoi_gdf.copy()
aoi_gdf_bounding_box['bounding_box'] = aoi_gdf_bounding_box.geometry.apply(lambda x: x.envelope)
aoi_gdf_bounding_box = aoi_gdf_bounding_box[['aoi', 'bounding_box']].rename(columns={'bounding_box': 'geometry'})

# AOI for Sao Paulo
aoi_saopaulo_bounding_box = aoi_gdf_bounding_box[aoi_gdf_bounding_box['aoi'] == "SaoPaulo"]
aoi_saopaulo_bounding_box = aoi_gdf_bounding_box.iloc[0:1]
aoi_saopaulo_bounding_box.reset_index(drop=True, inplace=True)

# AOI for Okha
aoi_wadden_okha_bounding_box = aoi_gdf_bounding_box[aoi_gdf_bounding_box['aoi'] == "Okha"]
aoi_wadden_okha_bounding_box = aoi_gdf_bounding_box.iloc[3:5]
aoi_wadden_okha_bounding_box.reset_index(drop=True, inplace=True)


c:\Users\fuentesm\AppData\Local\anaconda3\envs\copernicus\Lib\site-packages\xarray\core\dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
c:\Users\fuentesm\AppData\Local\anaconda3\envs\copernicus\Lib\site-packages\xarray\core\dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'p:\\11209821-cmems-global-sdb\\00_miscellaneous\\AOIs'

### Harmonize input data

In [6]:
depthmsl_ds = assign_projection(depthmsl_ds)
lat_ds = assign_projection(lat_ds)
lw_ds = assign_projection(lw_ds, 4326)
gebco_ds = assign_projection(gebco_ds, 4326)

print_ds_properties(depthmsl_rds)
print_ds_properties(lat_rds)
print_ds_properties(lw_rds, 4326)
print_ds_properties(gebco_rds, 4326)

lw_rds.rio.reproject_match(depthmsl_rds)

depthmsl_rds = depthmsl_rds.rio.set_nodata(np.nan)
lat_rds      = lat_rds.rio.set_nodata(np.nan)
lw_rds       = lat_rds.rio.set_nodata(np.nan)
gebco_rds    = lat_rds.rio.set_nodata(np.nan)

print_ds_properties(depthmsl_rds)
print_ds_properties(lat_rds)
print_ds_properties(lw_rds, 4326)
print_ds_properties(gebco_rds, 4326)

depthmsl_rds.rio.to_raster(os.path.join(output_data_path,'depthmsl_rds.tif'), driver='GTiff', compress='lzw')
lat_rds.rio.to_raster(os.path.join(output_data_path,'lat_rds.tif'), driver='GTiff', compress='lzw')
lw_rds.rio.to_raster(os.path.join(output_data_path,'lw_rds.tif'), driver='GTiff', compress='lzw')
gebco_rds.rio.to_raster(os.path.join(output_data_path,'gebco_rds.tif'), driver='GTiff', compress='lzw')

# If the harmonized data was already created then they can just loaded
# depthmsl_rds = rioxarray.open_rasterio(os.path.join(output_data_path,'depthmsl_rds.tif'))
# lat_rds = rioxarray.open_rasterio(os.path.join(output_data_path,'lat_rds.tif'))
# lw_rds = rioxarray.open_rasterio(os.path.join(output_data_path,'lw_rds.tif'))
# gebco_rds = rioxarray.open_rasterio(os.path.join(output_data_path,'gebco_rds.tif'))

EPSG:4326 projection was assigned to the dataset attributes
EPSG:4326 projection was assigned to the dataset attributes
EPSG:4326 projection was assigned to the dataset attributes
EPSG:4326 projection was assigned to the dataset attributes
Grid size: (0.004166666666666666, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666666666666, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666501832161, -0.004166666690214726)
no data: 255
There is no projection
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666666666666, -0.004166666666666667)
no data: -32767
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666666666666, -0.004166666666666667)
no data: nan
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666666666666, -0.004166666666666667)
no data: nan
The

### Generate Intertidal Zone

In [7]:
lat_mask = apply_lat_mask(depthmsl_rds, lat_rds)
lat_mask_gebco_plus_10 = apply_gebco_mask(gebco_rds,lat_mask)

lat_mask.rio.to_raster(os.path.join(output_data_path,"lat_mask.tif"))
lat_mask_gebco_plus_10.rio.to_raster(os.path.join(output_data_path,"lat_mask_gebco_plus_10.tif"))

# If the masked data was already created then they can just loaded
# lat_mask = rioxarray.open_rasterio(os.path.join(output_data_path,"lat_mask.tif"))
# lat_mask_gebco_plus_10 = rioxarray.open_rasterio(os.path.join(output_data_path,"lat_mask_gebco_plus_10.tif"))

### Clip results

In [8]:
depthmsl_rds_clipped_wadden_okha = clip_raster_with_gdf(depthmsl_rds, aoi_wadden_okha_bounding_box)
lat_rds_clipped_wadden_okha = clip_raster_with_gdf(lat_rds, aoi_wadden_okha_bounding_box)
lw_rds_clipped_wadden_okha = clip_raster_with_gdf(lw_rds, aoi_wadden_okha_bounding_box)
gebco_rds_clipped_wadden_okha = clip_raster_with_gdf(gebco_rds, aoi_wadden_okha_bounding_box)

lat_mask_gebco_plus_10.rio.set_crs("epsg:4326", inplace=True)
lat_mask_gebco_plus_10_clipped_wadden_okha = clip_raster_with_gdf(lat_mask_gebco_plus_10, aoi_wadden_okha_bounding_box)

lat_rds_clipped_wadden_okha[0].rio.to_raster(os.path.join(output_data_path,'lat_rds_clipped_wadden.tif'), driver='GTiff', compress='lzw')
lat_rds_clipped_wadden_okha[1].rio.to_raster(os.path.join(output_data_path,'lat_rds_clipped_okha.tif'), driver='GTiff', compress='lzw')
depthmsl_rds_clipped_wadden_okha[0].rio.to_raster(os.path.join(output_data_path,'depthmsl_rds_clipped_wadden.tif'), driver='GTiff', compress='lzw')
depthmsl_rds_clipped_wadden_okha[1].rio.to_raster(os.path.join(output_data_path,'depthmsl_rds_clipped_okha.tif'), driver='GTiff', compress='lzw')
lw_rds_clipped_wadden_okha[0].rio.to_raster(os.path.join(output_data_path,'lw_rds_clipped_wadden.tif'), driver='GTiff', compress='lzw')
lw_rds_clipped_wadden_okha[1].rio.to_raster(os.path.join(output_data_path,'lw_rds_clipped_okha.tif'), driver='GTiff', compress='lzw')
gebco_rds_clipped_wadden_okha[0].rio.to_raster(os.path.join(output_data_path,'gebco_rds_clipped_wadden.tif'), driver='GTiff', compress='lzw')
gebco_rds_clipped_wadden_okha[1].rio.to_raster(os.path.join(output_data_path,'gebco_rds_clipped_okha.tif'), driver='GTiff', compress='lzw')

lat_mask_gebco_plus_10_clipped_wadden_okha[0].rio.to_raster(os.path.join(output_data_path,'lat_mask_gebco_plus_10_clipped_wadden.tif'), driver='GTiff', compress='lzw')
lat_mask_gebco_plus_10_clipped_wadden_okha[1].rio.to_raster(os.path.join(output_data_path,'lat_mask_gebco_plus_10_clipped_okha.tif'), driver='GTiff', compress='lzw')

Successful processing row 0 WaddenSea
Successful processing row 1 Okha


Successful processing row 0 WaddenSea
Successful processing row 1 Okha


Successful processing row 0 WaddenSea
Successful processing row 1 Okha


Successful processing row 0 WaddenSea
Successful processing row 1 Okha


Successful processing row 0 WaddenSea
Successful processing row 1 Okha




### Generate polygons


In [18]:
lat_mask_gebco_plus_10_clipped_wadden = rioxarray.open_rasterio(os.path.join(output_data_path,'lat_mask_gebco_plus_10_clipped_wadden.tif'))

lat_mask_gebco_plus_10_clipped_wadden

binary_mask = lat_mask_gebco_plus_10_clipped_wadden > 8
lat_mask_gebco_plus_10_clipped_wadden_masked = lat_mask_gebco_plus_10_clipped_wadden.where(~binary_mask, other=np.nan)

lat_mask_gebco_plus_10_clipped_wadden_masked.rio.to_raster(os.path.join(output_data_path,"lat_mask_gebco_plus_10_clipped_wadden_masked.tif"))


In [19]:
lat_mask_gebco_plus_10_clipped_wadden_masked

<xarray.DataArray (band: 1, y: 449, x: 640)> Size: 1MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ...,  7.,  7.,  7.],
        [nan, nan, nan, ...,  7.,  7.,  7.],
        [nan, nan, nan, ...,  7.,  7.,  7.]]], dtype=float32)
Coordinates:
  * band         (band) int32 4B 1
  * x            (x) float64 5kB 5.185 5.19 5.194 5.198 ... 7.84 7.844 7.848
  * y            (y) float64 4kB 55.5 55.5 55.49 55.49 ... 53.64 53.64 53.64
    spatial_ref  int32 4B 0
Attributes:
    AREA_OR_POINT:      Area
    lat#long_name:      longitude
    lat#standard_name:  longitude
    lat#units:          degrees_east
    lon#long_name:      latitude
    lon#standard_name:  latitude
    lon#units:          degrees_north
    long_name:          DEPTH
    title:              Depth relative to MSL for GEBCO 2023 bathymetry
    _FillValue:         9999.0
    scale_factor:       1.0
    add_offset:         0.0

In [26]:
# Open raster file using rasterio
with rasterio.open(os.path.join(output_data_path,"lat_mask_gebco_plus_10_clipped_wadden_masked.tif")) as src:
    # Read raster data into numpy array
    raster_array = src.read(1)  # Assuming it's a single band raster, adjust if necessary
    # Extract transformation metadata
    transform = src.transform
    # Polygonize raster data
    polygons = list(shapes(raster_array, mask=None, transform=transform))
    # Convert polygons to Shapely geometries
    geometries = [shape(polygon) for polygon, _ in polygons]

# Convert Shapely geometries to GeoDataFrame
geo_df = gpd.GeoDataFrame(geometry=geometries)

# Define the EPSG code for the desired projection
epsg_code = 4326  # For example, EPSG code for WGS 84

# Assign the projection to the GeoDataFrame
geo_df.crs = f"EPSG:{epsg_code}"

# Save GeoDataFrame to file
geo_df.to_file(os.path.join(output_data_path,"lat_mask_gebco_plus_10_clipped_wadden_masked.shp"))
geo_df.to_file(os.path.join(output_data_path,"lat_mask_gebco_plus_10_clipped_wadden_masked.geojson"), driver="GeoJSON", crs=f"EPSG:{epsg_code}")

In [36]:
# Open raster file using rasterio
with rasterio.open(os.path.join(output_data_path,"lat_mask_gebco_plus_10_clipped_wadden_masked.tif")) as src:
    # Read raster data into numpy array
    raster_array = src.read(1)  # Assuming it's a single band raster, adjust if necessary
    # Extract transformation metadata
    transform = src.transform
    # Polygonize raster data
    polygons = list(shapes(raster_array, mask=None, transform=transform))
    # Convert polygons to Shapely geometries and record pixel values
    geometries_with_values = [(shape(polygon), value) for polygon, value in polygons]

# Extract geometries and values into separate lists
geometries = [geometry for geometry, value in geometries_with_values]
values = [value for geometry, value in geometries_with_values]

# Convert Shapely geometries and pixel values to GeoDataFrame
geo_df = gpd.GeoDataFrame(geometry=geometries, data={'pixel_value': values})

geo_df = geo_df[~np.isnan(geo_df['pixel_value'])]
geo_df.reset_index(drop=True, inplace=True)

# Define the EPSG code for the desired projection
epsg_code = 4326  # For example, EPSG code for WGS 84

# Assign the projection to the GeoDataFrame
geo_df.crs = f"EPSG:{epsg_code}"

# Save GeoDataFrame to file
geo_df.to_file(os.path.join(output_data_path,"lat_mask_gebco_plus_10_clipped_wadden_masked.shp"))
geo_df.to_file(os.path.join(output_data_path,"lat_mask_gebco_plus_10_clipped_wadden_masked.geojson"), driver="GeoJSON", crs=f"EPSG:{epsg_code}")


C:\Users\fuentesm\AppData\Local\Temp\ipykernel_11216\4154894453.py:29: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geo_df.to_file(os.path.join(output_data_path,"lat_mask_gebco_plus_10_clipped_wadden_masked.shp"))
